<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Main" data-toc-modified-id="Main-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Main</a></span></li></ul></div>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Main" data-toc-modified-id="Main-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Main</a></span></li></ul></div>

In [ ]:
# default_exp exec.train_vits

In [ ]:
# export
import argparse
import json
import librosa  # NOTE(zach): importing torch before librosa causes LLVM issues for some unknown reason.
import sys

import torch
from torch import multiprocessing as mp

from uberduck_ml_dev.trainer.vits import VITSTrainer
from uberduck_ml_dev.vendor.tfcompat.hparam import HParams
from uberduck_ml_dev.models.vits import DEFAULTS as VITS_DEFAULTS


def parse_args(args):
    parser = argparse.ArgumentParser()
    parser.add_argument("--config", help="Path to JSON config")
    args = parser.parse_args(args)
    return args

In [ ]:
assert isinstance(VITS_DEFAULTS, HParams)
assert parse_args(["--config", "foo.json"]).config == "foo.json"
assert parse_args([]).config is None
assert HParams(**VITS_DEFAULTS.values()).values() == VITS_DEFAULTS.values()

In [ ]:
# export
def run(rank, device_count, hparams):
    trainer = VITSTrainer(hparams, rank=rank, world_size=device_count)
    try:
        trainer.train()
    except Exception as e:
        print(f"Exception raised while training: {e}")
        # TODO: save state.
        raise e

# Main

In [ ]:
# export
try:
    from nbdev.imports import IN_NOTEBOOK
except:
    IN_NOTEBOOK = False
if __name__ == "__main__" and not IN_NOTEBOOK:
    args = parse_args(sys.argv[1:])
    config = VITS_DEFAULTS.values()
    if args.config:
        with open(args.config) as f:
            config.update(json.load(f))
    hparams = HParams(**config)
    if hparams.distributed_run:
        device_count = torch.cuda.device_count()
        mp.spawn(run, (device_count, hparams), device_count)
    else:
        run(0, 1, hparams)